In [2]:
from random import random
from numpy import array
from numpy import cumsum

In [4]:
def get_sequence(n_timesteps):
    X=array([random() for _ in range(n_timesteps)])
    limit = n_timesteps/4.0
    y = array([0 if x<limit else 1 for x in cumsum(X)])
    return X, y

In [5]:
X,y = get_sequence(10)
print(X)
print(y)

[0.91625924 0.93891272 0.48190771 0.80068555 0.04311927 0.72207417
 0.97856538 0.57380721 0.93697502 0.63398351]
[0 0 0 1 1 1 1 1 1 1]


In [6]:
def get_sequences(n_sequences,n_timesteps):
    seqX, seqY = list(), list()
    for _ in range(n_sequences):
        X,y = get_sequence(n_timesteps)
        seqX.append(X)
        seqY.append(y)
    seqX = array(seqX).reshape(n_sequences,n_timesteps,1)
    seqY = array(seqY).reshape(n_sequences,n_timesteps,1)
    return seqX, seqY

In [9]:
n_timesteps=10

In [7]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Bidirectional(LSTM(50,return_sequences=True),input_shape=(n_timesteps,1)))
model.add(TimeDistributed(Dense(1,activation='sigmoid')))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
print(model.summary())

W0814 17:47:13.517924 140382590072640 module_wrapper.py:136] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 17:47:13.546528 140382590072640 deprecation.py:323] From /home/arvind/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
X,y = get_sequences(1000,n_timesteps)
model.fit(X,y,epochs=1,batch_size=10)

Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3558 - acc: 0.8862


In [12]:
X,y = get_sequences(100,n_timesteps)
loss, acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.196616, Accuracy: 91.599999


In [13]:
from numpy import array_equal
for _ in range(10):
    X,y = get_sequences(1,n_timesteps)
    yhat = model.predict_classes(X,verbose=0)
    exp, pred = y.reshape(n_timesteps),yhat.reshape(n_timesteps)
    print('y=%s, yhat=%s, correct=%s' %(exp,pred,array_equal(exp,pred)))

y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 0 0 0 0 0 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
